In [ ]:
###########################################################################################
# Written for AIFS
# MODULE 3A - Geometric Raytracing
# Copyright 2021 Tarek Zohdi, Emre Mengi. All rights reserved.
###########################################################################################

### Module 5A - Basics of Geometric Raytracing

Written for AIFS.   
Copyright 2023 Tarek Zohdi, Emre Mengi. All rights reserved.

In this assignment, you will be introduced to the basics of light-based simulation methods.

In [1]:
############################################### Import Packages ##########################################
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import time
import csv
import os

### **Problem 1:** Theory-Based Exercises ###

Answer the following questions *prior* to coding the assignment to better understand the background physics and mathematics that govern the given models. You **may** solve these problems by hand **and/or** using computational tools such as *Python* etc. Please include all handwritten work and code used to solve each problem.

**Problem 1.2:** Write down the Forward Euler equation for time discretization. Explain all the terms.

*Answer here*

### **Problem 2:** Coding Exercises ###  
Use the given python notebook template to complete the following coding exercises.

**Problem 2.1:** Define the constants used in the simulation. Use the variable glossary at the end of the assignment.

In [4]:
################################ Define Constants ##########################################

mu_0 = 4*np.pi*1e-7 #free-space magn. permeability [Wb/(Am)]
eps_0 = 8.8542*1e-12 #free-space elec. permittivity [C/(Nm)]
c = 1/np.sqrt(mu_0*eps_0) #speed of light [m/s]
#mu_hat: ratio of magn. permeability scattering/absorbing (mu_a)...
mu_hat = 1
eps_hat = 25; # electric permitivity (chosen)
#n_hat = np.sqrt(mu_hat * eps_hat)
n_hat = 10

In [6]:
# Set up the surface
L1 = 1;
L2 = 1;
w1 = 1.5;
w2 = 0.75;
A = 0.3;
test_lim = 0.5; #limits of test surface
x1 = np.linspace(-test_lim,test_lim,100)
x2 = np.linspace(-test_lim,test_lim,100)

[X1, X2] = np.meshgrid(x1,x2)
X1 = np.reshape(X1,[np.size(X1),1])
X2 = np.reshape(X2,[np.size(X2),1])


X3 = 1.5 + A * np.sin(2 * w1 * np.pi * X1 / L1) * np.sin(2 * w2 * np.pi * X2 / L2) # surface a
for i in range(X3.size): #uncomment for surface a
    if X3[i]-2 < 0:
        X3[i] = 2
# X3 = np.sqrt((5 - np.sqrt(X1**2 + X2**2))**2 - 4)-1.75 # surface b



      
# Calculate normals
#Manual Method

#surface A
#delF
delGx1 = A*(2*w1*np.pi/L1)*np.cos(2*w1*np.pi*X1/L1)*np.sin(2*w2*np.pi*X2/L2);
delGx2 = A*(2*w2*np.pi/L2)*np.sin(2*w1*np.pi*X1/L1)*np.cos(2*w2*np.pi*X2/L2);

delFx1 = delGx1
delFx2 = delGx2
delFx3 = np.ones([np.size(delGx1),1])*-1

gradF = np.hstack([delFx1, delFx2, delFx3])
magdelF = np.reshape(np.linalg.norm(gradF,2,1),(np.size(X1),1))

normal = - gradF / magdelF

for i in range(X3.size): #uncomment for surface a NORMAL + SURFACE POINT FIX
    if X3[i]-2 < 0:
        X3[i] = 2
        normal[i,0] = 0
        normal[i,1] = 0
        normal[i,2] = 1
        
#surface A
#delF
# delGx1 = 1/(2*np.sqrt((5 - np.sqrt(X1**2 + X2**2))**2 - 4))*2*(5 - np.sqrt(X1**2 + X2**2))*-1/(2*np.sqrt(X1**2 + X2**2))*2*X1
# delGx2 = 1/(2*np.sqrt((5 - np.sqrt(X1**2 + X2**2))**2 - 4))*2*(5 - np.sqrt(X1**2 + X2**2))*-1/(2*np.sqrt(X1**2 + X2**2))*2*X2

delFx1 = delGx1
delFx2 = delGx2
delFx3 = np.ones([np.size(delGx1),1])*-1

gradF = np.hstack([delFx1, delFx2, delFx3])
magdelF = np.reshape(np.linalg.norm(gradF,2,1),(np.size(X1),1))

normal = - gradF / magdelF

# for i in range(X3.size): #uncomment for surface a NORMAL + SURFACE POINT FIX
#     if X3[i]-1.5 < 0:
#         X3[i] = 1.5
#         normal[i,0] = 0
#         normal[i,1] = 0
#         normal[i,2] = 1
   

# Save particle data

if os.path.exists("rad1.dat"):
    os.remove("rad1.dat")
    # Print the statement once
    # the file is deleted 
    print("Previous file deleted.") 

with open('rad1.dat', 'a') as p_file:
    writer = csv.writer(p_file, delimiter='\t')  # make write variable
    writer.writerow(['TITLE = ', 'Light-scatter Simulation'])
    writer.writerow(['VARIABLES = ', 'X-coord', 'Y-coord', 'Z-coord', 'X-vec', 'Y-vec', 'Z-vec'])  # headings
    writer.writerow(['ZONE'])  # self explanatory
    for p in range(np.size(X1)):  # this for-loop writes each particle's (row) data
      writer.writerow([X1[p,0],X2[p,0],X3[p,0],normal[p,0],normal[p,1],normal[p,2]])


# Create rays
beam_initvel = np.array([0,0,-c]) #initial velocity of beam
#beam_initvel = [0,c/np.sqrt(2),-c/np.sqrt(2)] #initial velocity of beam
beam_N = 1000 #number of rays
beam_initH = 3 #initial height
beam_N = round(np.sqrt(beam_N))**2; # updated number of rays to have it square
print('Updated number of rays is:',beam_N)

#Different beam types
#full beam
[beam_posx1,  beam_posx2] = np.meshgrid(np.linspace(-0.5,0.5,round(np.sqrt(beam_N))),np.linspace(-0.5,0.5,round(np.sqrt(beam_N))));
beam_posx1 = np.reshape(beam_posx1,[np.size(beam_posx1),1])
beam_posx2 = np.reshape(beam_posx2,[np.size(beam_posx2),1])

##center beam (square)
#[beam_posx1,  beam_posx2] = np.meshgrid(np.linspace(-0.25,0.25,round(np.sqrt(beam_N))),np.linspace(-0.25,0.25,round(np.sqrt(beam_N))));
# beam_posx1 = np.reshape(beam_posx1,[np.size(beam_posx1),1])
# beam_posx2 = np.reshape(beam_posx2,[np.size(beam_posx2),1])

##offset beam
#beam_posx2 = beam_posx2 - 1;

beam_posx3 = np.ones([beam_N,1]) * beam_initH;
beam_vel1 = np.ones([beam_N,1])  * beam_initvel[0];
beam_vel2 = np.ones([beam_N,1])  * beam_initvel[1];
beam_vel3 = np.ones([beam_N,1])  * beam_initvel[2];

# Time-stepping
totaltime = 0.0000000075 # simulation time
xi = 0.00000005 # time-stepping constant
#xi = 0.000000005 # time-stepping constant
dt = xi * beam_initH / np.sqrt(c)
Ntimestep = round(totaltime / dt)

IR = np.zeros([np.size(beam_posx1),1])
theta_i = np.zeros([np.size(beam_posx1),1])
X1_final = np.empty([np.size(beam_posx1),1])
X2_final = np.empty([np.size(beam_posx1),1])
X3_final = np.empty([np.size(beam_posx1),1])

if os.path.exists("rad2.dat"):
    os.remove("rad2.dat")
    # Print the statement once
    # the file is deleted 
    print("Previous file deleted.") 
    
if os.path.exists("rad3.dat"):
    os.remove("rad3.dat")
    # Print the statement once
    # the file is deleted 
    print("Previous file deleted.") 

for i in range(Ntimestep):
    
    #time-step announcer
    print(i+1,' of ',Ntimestep,'th time step.')
    
    beam_posx1 = beam_posx1 + beam_vel1 * dt;
    beam_posx2 = beam_posx2 + beam_vel2 * dt;
    beam_posx3 = beam_posx3 + beam_vel3 * dt;
     
    for k in range(np.size(beam_posx1)):
        #check if below surface
        X3_calc = 1.5 + A*np.sin(2*w1*np.pi*beam_posx1[k]/L1)*np.sin(2*w2*np.pi*beam_posx2[k]/L2); #surface a
        for z in range(np.size(X3_calc)): #uncomment for surface a
            if X3_calc-1.5 < 0:
                X3_calc = 1.5
#         X3_calc = np.sqrt((5 - np.sqrt(beam_posx1[k]**2 + beam_posx2[k]**2))**2 - 4) - 1.75; #surface b

        if beam_posx1[k] <= test_lim and beam_posx1[k] >= -test_lim and beam_posx2[k] <= test_lim and beam_posx2[k] >= -test_lim:
           if X3_calc>beam_posx3[k]:
               if IR[k] == 0:
                    #find normal and ray velocity vector
                    # Calculate normals
                    ##Manual Method
                    #delF
                    #surface a
                    delGx1 = A*(2*w1*np.pi/L1)*np.cos(2*w1*np.pi*beam_posx1[k]/L1)*np.sin(2*w2*np.pi*beam_posx2[k]/L2);
                    delGx2 = A*(2*w2*np.pi/L2)*np.sin(2*w1*np.pi*beam_posx1[k]/L1)*np.cos(2*w2*np.pi*beam_posx2[k]/L2);
                    
#                     #surface b
#                     delGx1 = 1/(2*np.sqrt((5 - np.sqrt(beam_posx1[k]**2 + beam_posx2[k]**2))**2 - 4))*2*(5 - np.sqrt(beam_posx1[k]**2 + beam_posx2[k]**2))*-1/(2*np.sqrt(beam_posx1[k]**2 + beam_posx2[k]**2))*2*beam_posx1[k]
#                     delGx2 = 1/(2*np.sqrt((5 - np.sqrt(beam_posx1[k]**2 + beam_posx2[k]**2))**2 - 4))*2*(5 - np.sqrt(beam_posx1[k]**2 + beam_posx2[k]**2))*-1/(2*np.sqrt(beam_posx1[k]**2 + beam_posx2[k]**2))*2*beam_posx2[k]


                    
                    delFx1 = delGx1
                    delFx2 = delGx2
                    delFx3 = -1
                    
                    gradF = np.hstack([delFx1, delFx2, delFx3])
                    magdelF = np.linalg.norm(gradF,2)
                    
                    normal = - gradF / magdelF
                    
#                     # surface a
#                     if X3_calc == 1.5:
#                         normal[0] = 0
#                         normal[1] = 0
#                         normal[2] = 1
                            
                    norm_k = normal
                    beam_vel = np.hstack([beam_vel1[k], beam_vel2[k], beam_vel3[k]])
                    theta_i = np.arccos(np.dot(norm_k,beam_vel)/c)
                    if beam_vel3[k] <= 0:
                        #record point of contact with surface
                        X1_final[k] = beam_posx1[k] 
                        X2_final[k] = beam_posx2[k]
                        X3_final[k] = beam_posx3[k]
                    
                        beam_vel = beam_vel - 2 * c * np.cos(theta_i) * norm_k;
                        beam_vel1[k] = beam_vel[0];
                        beam_vel2[k] = beam_vel[1];
                        beam_vel3[k] = beam_vel[2];
                        #theta_i = np.pi - theta_i
                        IR[k] = (np.abs(theta_i/n_hat) <= 1.)* \
                            (0.5*(((n_hat**2 * np.cos(theta_i) - (n_hat**2 - np.sin(theta_i)**2)**(0.5))/ \
                                    (n_hat**2 * np.cos(theta_i) + (n_hat**2 - np.sin(theta_i)**2)**(0.5)))**2 +\
                                   (((np.cos(theta_i) - (n_hat**2 - np.sin(theta_i)**2)**(0.5))/ \
                                     (np.cos(theta_i) + (n_hat**2 - np.sin(theta_i)**2)**(0.5)))**2)))+ \
                                (np.abs(theta_i/n_hat) > 1.)**1.;



    if i % 5  == 0:
        with open('rad2.dat', 'a') as p_file:
            if i == 0:
                writer = csv.writer(p_file, delimiter='\t')  # make write variable
                writer.writerow(['TITLE = ', 'Light-scatter Simulation'])
                writer.writerow(['VARIABLES = ', 'X-coord', 'Y-coord', 'Z-coord','X-vel', 'Y-vel', 'Z-vel'])  
                # headings
            writer = csv.writer(p_file, delimiter=' ')  # make write variable
            writer.writerow(['ZONE'])
            writer.writerow(['STRANDID=',i+1])
            writer.writerow(['SOLUTIONTIME=',(i+1)*dt])
            #assign current loc, vel, and energy left in rays in seperate arrays
            xpos = beam_posx1
            ypos = beam_posx2
            zpos = beam_posx3
            xvel = beam_vel1
            yvel = beam_vel2
            zvel = beam_vel3
            writer = csv.writer(p_file, delimiter='\t')  # make write variable
            for p in range(np.size(xpos)):  # this for-loop writes each particle's (row) data
                writer.writerow([float(xpos[p]),float(ypos[p]),float(zpos[p]),float(xvel[p]),float(yvel[p]),float(zvel[p])])
                
        with open('rad3.dat', 'a') as p_file:
            if i == 0:
                writer = csv.writer(p_file, delimiter='\t')  # make write variable
                writer.writerow(['TITLE = ', 'Light-scatter Simulation'])
                writer.writerow(['VARIABLES = ', 'X-coord', 'Y-coord', 'Z-coord','Absorption'])  
                # headings
            writer = csv.writer(p_file, delimiter=' ')  # make write variable
            writer.writerow(['ZONE'])
            writer.writerow(['STRANDID=',i+1])
            writer.writerow(['SOLUTIONTIME=',(i+1)*dt])
            #assign current loc, vel, and energy left in rays in seperate arrays
            writer = csv.writer(p_file, delimiter='\t')  # make write variable
            for p in range(np.size(X1_final)):  # this for-loop writes each particle's (row) data
                if math.isnan(X1_final[p]):
                    pass
                else:
                    writer.writerow([float(X1_final[p]),float(X2_final[p]),float(X3_final[p]),float(IR[p])])

Previous file deleted.
Updated number of rays is: 1024
Previous file deleted.
Previous file deleted.
1  of  866 th time step.
2  of  866 th time step.
3  of  866 th time step.
4  of  866 th time step.
5  of  866 th time step.
6  of  866 th time step.
7  of  866 th time step.
8  of  866 th time step.
9  of  866 th time step.
10  of  866 th time step.
11  of  866 th time step.
12  of  866 th time step.
13  of  866 th time step.
14  of  866 th time step.
15  of  866 th time step.
16  of  866 th time step.
17  of  866 th time step.
18  of  866 th time step.
19  of  866 th time step.
20  of  866 th time step.
21  of  866 th time step.
22  of  866 th time step.
23  of  866 th time step.
24  of  866 th time step.
25  of  866 th time step.
26  of  866 th time step.
27  of  866 th time step.
28  of  866 th time step.
29  of  866 th time step.
30  of  866 th time step.
31  of  866 th time step.
32  of  866 th time step.
33  of  866 th time step.
34  of  866 th time step.
35  of  866 th time step

306  of  866 th time step.
307  of  866 th time step.
308  of  866 th time step.
309  of  866 th time step.
310  of  866 th time step.
311  of  866 th time step.
312  of  866 th time step.
313  of  866 th time step.
314  of  866 th time step.
315  of  866 th time step.
316  of  866 th time step.
317  of  866 th time step.
318  of  866 th time step.
319  of  866 th time step.
320  of  866 th time step.
321  of  866 th time step.
322  of  866 th time step.
323  of  866 th time step.
324  of  866 th time step.
325  of  866 th time step.
326  of  866 th time step.
327  of  866 th time step.
328  of  866 th time step.
329  of  866 th time step.
330  of  866 th time step.
331  of  866 th time step.
332  of  866 th time step.
333  of  866 th time step.
334  of  866 th time step.
335  of  866 th time step.
336  of  866 th time step.
337  of  866 th time step.
338  of  866 th time step.
339  of  866 th time step.
340  of  866 th time step.
341  of  866 th time step.
342  of  866 th time step.
3

618  of  866 th time step.
619  of  866 th time step.
620  of  866 th time step.
621  of  866 th time step.
622  of  866 th time step.
623  of  866 th time step.
624  of  866 th time step.
625  of  866 th time step.
626  of  866 th time step.
627  of  866 th time step.
628  of  866 th time step.
629  of  866 th time step.
630  of  866 th time step.
631  of  866 th time step.
632  of  866 th time step.
633  of  866 th time step.
634  of  866 th time step.
635  of  866 th time step.
636  of  866 th time step.
637  of  866 th time step.
638  of  866 th time step.
639  of  866 th time step.
640  of  866 th time step.
641  of  866 th time step.
642  of  866 th time step.
643  of  866 th time step.
644  of  866 th time step.
645  of  866 th time step.
646  of  866 th time step.
647  of  866 th time step.
648  of  866 th time step.
649  of  866 th time step.
650  of  866 th time step.
651  of  866 th time step.
652  of  866 th time step.
653  of  866 th time step.
654  of  866 th time step.
6